Proteus FSI Workshop, Oxford 2017![triton](http://proteustoolkit.org/_static/corps_logo_cross2.svg)
=================

This notebook is also a (live) slide presentation. To view it as a presentation, click on the button in the toolbar that looks like this: ![rise button](rise.png)

Use the space bar to step through and shift-enter to execute code cells.

### Acknowledgements
---
- Math Institute: Niall Bootland and Chris Breward (Oxford)
- HR Wallingford: Aggelos Dimakopoulos
- ERDC: Natalie Elwart
- Funding:
  - [ERDC](http://www.erdc.usace.army.mil) 219 Program
  - [ERDC Military Engineering](http://www.erdc.usace.army.mil/Missions/Military-Engineering/)
  - [Oxford InFoMM](https://www.ox.ac.uk/admissions/graduate/courses/industrially-focused-mathematical-modelling?wssl=1)


### Logistics
---
- Facilities at the Math Institute
- Headcount and rides for Wednesday
- WiFi
- [Schedule: http://erdc-cm.github.io/sprints ](http://erdc-cm.github.io/sprints)
- [Wiki: https://github.com/erdc-cm/sprints/wiki/Proteus-FSI-Workshop-July-3-to-7](https://github.com/erdc-cm/sprints/wiki/Proteus-FSI-Workshop-July-3-to-7)

### Survey of recent R&D in Proteus

### Entropy Viscosity @mquezada
---
- Moving away from shock capturing while maintaining qualitative properties (e.g. bounds) and achieving higher order.
- Going into SWE and RANS3P
<video width="800" src="3Bumps_friction_v2.mp4" controls>
Your browser does not support the video tag
</video>

### Splitting Methods  @cekees
---
- Extending 2nd Order Splitting of [Guermond and Salgado 2009](http://www.math.tamu.edu/~guermond/PUBLICATIONS/guermond_salgado_jcp_2009.pdf) to support air/water and air/water/sediment flows (`RANS3P*`).
- Velocity update $$ \alpha \rho^{n+1} \vec{v}^{n+1} + \beta + \rho^{n+1}(\vec{v}^{\star}\cdot \nabla)\vec{v}^{n+1} - \mu^{n+1} \Delta \vec{v}^{n+1} + \nabla p^{\star} =  \vec{f}^{n+1}$$

- Correction $$ \Delta \phi^{n+1} = \frac{3 \chi}{2} \nabla \cdot \vec{v}^{n+1} $$

- Pressure update $$ p^{n+1} = p^n + \phi^{n+1} + \nabla \cdot \vec{v}^{n+1}$$
- Boundary condition (added mass...)

### Three-Phase Flow @gcozzuto
---
$$ \theta^s + \theta^a + \theta^w = 1 $$
$$\frac{\partial \left( \rho^{\alpha} \theta^{\alpha} \right)}{\partial t} + \nabla \cdot \left(
\rho^{\alpha} \theta^{\alpha} \vec{v}^{\alpha} \right)  = 0$$ 
$$ \frac{\partial \theta^s}{\partial t} + \nabla \cdot \left(
\theta^s \vec{v}^s \right)  = 0$$
$$ \frac{\partial \left( \epsilon S \right)}{\partial t} + \nabla  \cdot \left(
      \epsilon S \vec{v}^w \right) = 0 $$
$$ \rho^f \epsilon \frac{\partial \vec{v}^f }{ \partial t} +  \epsilon
\rho^f\vec{v}^f\cdot \nabla \vec{v}^f + \nabla  \cdot  \tau^f  + \epsilon
\nabla p^f - \epsilon\rho^f\vec{g} + \mathcal{I}^{fs} = 0 $$
$$ \rho^s \theta^s \frac{\partial \vec{v}^s }{ \partial t} +
\theta^s\rho^s\vec{v}^s \cdot \nabla \vec{v}^s  + \nabla \cdot \tau^s +
\theta^s \nabla p^f - \theta^s \rho^s \vec{g} - \mathcal{I}^{fs} = 0 $$
$$ \nabla \cdot \vec{v}^t = 0$$

### Moored Rigid Structures @tridelat
---
ALE coupling with Chrono, problem setup (`SpatialTools` and `BoundaryConditions`)
<center>
<video width="900" src="moored_cylinder.mp4" controls>
Your browser does not support the video tag
</video>
</center>

### Deforming Structures @milad-rakhsha
---
- Support rigid and deforming structures in Chrono via both ALE and IBM
<center>
<video height="250" src="ibm_plate.mp4" controls>
Your browser does not support the video tag
</video>
<video height="250" src="dambreak.mp4" controls>
Your browser does not support the video tag
</video>
</center>

### FEM-DEM Coupling @dtoconn
---
- Focus is on IBM linking proteus, mosaic, and chrono
<center>
<video height="500" src="free_cylinder.mp4" controls>
Your browser does not support the video tag
</video>
</center>

### Adaptivity @zhanga
---
Integration of proteus with RPI SCOREC, error estimates
<center>
<video height="600" src="adaptive.mp4" controls>
Your browser does not support the video tag
</video>
</center>

### Jupyter notebooks and apps @jamijares
---
- Computational flume and basin web apps vi Jupyter notebooks
- New ipython widget package by @cekees [ipymesh](https://github.com/erdc-cm/ipymesh)

In [2]:
from ipymesh import PSLGEditor
editor = PSLGEditor(vertices=[(0.1,0.1)],regionTypes=[1,2,0], boundaryTypes=[1,2,3,4,0], 
                    Lx=1.0, Ly=1.0, width=800, height=400)
editor

In [3]:
from ipymesh import PSLGEditor
from proteus import Domain
class PSLG_DomainEditor(PSLGEditor, Domain.PlanarStraightLineGraphDomain):
    def __init__(self, *args, **kwargs):
        super(PSLG_DomainEditor, self).__init__(*args, **kwargs)
domain_editor = PSLG_DomainEditor(vertices=[(0.1,0.1)],regionTypes=[1,2,0], boundaryTypes=[1,2,3,4,0], x0=0.0, y0=0.0,Lx=1.0, Ly=1.0, width=800, height=400, polyfile=None, regionConstraints=None)
domain_editor

In [4]:
from proteus import TriangleTools
domain_editor.polyfile=None
domain_editor.writePoly('mesh')
tmesh = TriangleTools.TriangleBaseMesh(baseFlags='VApq30Dena{0:f}'.format(0.001),
                                       nbase=1,
                                       verbose=10)
tmesh.readFromPolyFile(domain_editor.polyfile)
tmesh.writeToFile(domain_editor.polyfile)
mesh=tmesh.convertToProteusMesh(verbose=1)

In [5]:
import ipympl
from matplotlib import pyplot as plt
fig, ax = plt.subplots(1,1)
ax.tripcolor(mesh.nodeArray[:,0],
              mesh.nodeArray[:,1],
              mesh.elementNodesArray,
              edgecolors='k',
              facecolors=mesh.elementMaterialTypes)
fig.canvas

### Docker containers @cekees
---
- Public docker containers build automatically for master and release branches.
- [https://cloud.docker.com](https://cloud.docker.com/swarm/erdc/repository/docker/erdc/proteus/general)
```
docker pull erdc/proteus:latest
```
```
docker pull erdc/proteus:release-1.3.3```

- If you like the commandline then do something like
```
docker run -it erdc/proteus:latest bash```

- If you like to work through jupyter notebooks do something like
```
docker run -it --host=net erdc/proteus:latest```


###  Binder @cekees
---
- Enable public "binder" of repositories with Jupyter notebooks
- Just extend off of the proteus Docker image with a Dockerfile:
```
FROM erdc/proteus:latest
MAINTAINER Proteus <proteus-dev@groups.google.com>
COPY . $HOME
```
- **This** notebook [https://github.com/erdc-cm/oxford2017](https://github.com/erdc-cm/oxford2017)

### Portable binaries, hashdist/conda integration @msarahan
---
- The Hashdist tool and stack we depend on is being integrated with the popular conda tool and anaconda stack 
- Support for `conda install proteus` on linux, mac, and windows into anaconda Python
- Support for `conda install pkg` inside proteus stack just like `pip install` works now
- Improved support for portable binaries and conda channels with proteus stack packages.

### New licensing @cekees
---
- Moving from NASA license to standard MIT [LICENSE](https://github.com/erdc-cm/proteus/blob/update_dev/LICENSE.md)
- Thanks to strategy pioneered by [US Defense Digital Service](https://www.dds.mil/)
- Add your name and copyright (if eligible) to [CONTRIBUTORS](https://github.com/erdc-cm/proteus/blob/update_dev/CONTRIBUTORS.md)
- See also [CONTRIBUTING](https://github.com/erdc-cm/proteus/blob/update_dev/CONTRIBUTING.md) and [RELEASE](https://github.com/erdc-cm/proteus/blob/update_dev/RELEASE.md)

### Workshop Objectives
---
- Continuum air/water/sediment models
- Immersed Boundary Methods
- Meshing (generation, adaptivity, etc.)
- FEM-DEM approaches to granular media 
- Preconditioners for multi-phase Navier-Stokes equations
- Galerkin finite element methods for NSE and SWE
- Proteus 1.4 and 2.0 plan

### Scaling and "10x" development
---
- Steve McConnell, author of [Code Complete](https://www.amazon.co.uk/Code-Complete-Practical-Handbook-Construction/dp/0735619670), talks about factor of 10 difference in productivity for individuals and teams
- Lots of good and bad discussion about 10x developers, whether they exist, etc.
- Interesting blog post by Jessic Kerr [http://blog.jessitron.com/2017/06/the-most-productive-circumstances-for.html](http://blog.jessitron.com/2017/06/the-most-productive-circumstances-for.html)
  - "it isn't the developers, so much as the situation"
  - "The system is alive in our head"
  - "Please write tests"
  - "Pair program!"
  
TL;DR: the most productive development happens when one person knows the system intimately because they wrote it; this is in conflict with growing a system beyond what one person maintains.



### Programming "Pairs"
---
- Threephase: **Giovanni**, Aggelos, Chris, Manuel
- FEM-DEM: **Devin**, Chris, Manuel, (Arnold, Milad)
- Meshing: **Tristan**, **Alvin**, Chris, (Amanda, Grace, Jorge)
- Preconditioning: **Alistair**, Niall, Chris
- Galerkin Methods: **Manuel**, Chris, Alvin, Alistair (Matthew)
- Boundary Conditions: **Helen**, Giovanni, Aggelos, Chris
- Releases: **Chris**, **Aggelos**, ALL...

###  Discussion